In [26]:
import cv2
from PIL import Image
import numpy as np
def readFlow(fn):
    """ Read .flo file in Middlebury format"""
    # Code adapted from:
    # http://stackoverflow.com/questions/28013200/reading-middlebury-flow-files-with-python-bytes-array-numpy

    # WARNING: this will work on little-endian architectures (eg Intel x86) only!
    # print 'fn = %s'%(fn)
    with open(fn, 'rb') as f:
        magic = np.fromfile(f, np.float32, count=1)
        if 202021.25 != magic:
            print('Magic number incorrect. Invalid .flo file')
            return None
        else:
            w = np.fromfile(f, np.int32, count=1)
            h = np.fromfile(f, np.int32, count=1)
            # print 'Reading %d x %d flo file\n' % (w, h)
            data = np.fromfile(f, np.float32, count=2 * int(w) * int(h))
            # Reshape testdata into 3D array (columns, rows, bands)
            # The reshape here is for visualization, the original code is (w,h,2)
            return np.resize(data, (int(h), int(w), 2))
def path_to_img_arr(path,cv2_ = True):
  """  if cv2_:
    image = cv2.imread(path)
  else:"""
  image = Image.open(path).convert('RGB')
  image_array = np.asarray(image)
  return image_array

def warp(im, flow, alpha=1, interp=cv2.INTER_CUBIC):
    height, width, _ = flow.shape
    cart = np.dstack(np.meshgrid(np.arange(width), np.arange(height)))
    pixel_map = (cart + alpha * flow).astype(np.float32)
    warped = cv2.remap(
        im,
        pixel_map[:, :, 0],
        pixel_map[:, :, 1],
        interp)
    return warped

import cv2
def warp2(curImg ,flow):
  h, w = flow.shape[:2]
  flow = -flow
  flow[:,:,0] += np.arange(w)
  flow[:,:,1] += np.arange(h)[:,np.newaxis]
  prevImg = cv2.remap(curImg, flow, None, cv2.INTER_LINEAR)
  return prevImg
def warp3(img,flo):
  batched_img = np.reshape(img, (1, *img.shape))
  torch_img = torch.from_numpy(batched_img)
  torch_img = torch.from_numpy(np.transpose(batched_img, (0, 3, 1, 2)))
  batched_flo = np.reshape(flo, (1, *flo.shape))
  torch_flo = torch.from_numpy(batched_flo)
  torch_flo = torch.from_numpy(np.transpose(batched_flo, (0, 3, 1, 2)))
  return warp_pwc(torch_img,torch_flo)

def warp_pwc(x, flo):
    """
    warp an image/tensor (im2) back to im1, according to the optical flow

    x: [B, C, H, W] (im2)
    flo: [B, 2, H, W] flow

    """
    B, C, H, W = x.size()
    # mesh grid
    xx = torch.arange(0, W).view(1,-1).repeat(H,1)
    yy = torch.arange(0, H).view(-1,1).repeat(1,W)
    xx = xx.view(1,1,H,W).repeat(B,1,1,1)
    yy = yy.view(1,1,H,W).repeat(B,1,1,1)
    grid = torch.cat((xx,yy),1).float()

    if x.is_cuda:
        grid = grid.cuda()
    vgrid = Variable(grid) + flo

    # scale grid to [-1,1]
    vgrid[:,0,:,:] = 2.0*vgrid[:,0,:,:].clone() / max(W-1,1)-1.0
    vgrid[:,1,:,:] = 2.0*vgrid[:,1,:,:].clone() / max(H-1,1)-1.0

    vgrid = vgrid.permute(0,2,3,1)
    output = nn.functional.grid_sample(x, vgrid)
    mask = torch.autograd.Variable(torch.ones(x.size())).cuda()
    mask = nn.functional.grid_sample(mask, vgrid)

    mask[mask<0.999] = 0
    mask[mask>0] = 1

    return output*mask

def diff_show(path1 , path2):

    i_ref=path1
    i_mod=path2
    plt.figure(figsize=(5,5))
    image_ref = cv2.imread(i_ref)
    plt.imshow(image_ref[:, :, ::-1])
    plt.axis('off')
    plt.show()
    plt.figure(figsize=(5,5))
    image_mod = cv2.imread(i_mod)
    plt.imshow(image_mod[:, :, ::-1])
    plt.axis('off')
    plt.show()
    plt.figure(figsize=(5,5))
    plt.imshow(cv2.cvtColor(image_mod, cv2.COLOR_BGR2GRAY) - cv2.cvtColor(image_ref, cv2.COLOR_BGR2GRAY))
    plt.axis('off')
    plt.show()

    plt.close()

In [22]:
images = "../../sources/images"
flow_res = "../../sources/flows"
mod = images + "/demo_mod"
ref = images + "/demo_ref"

In [24]:
!ls $flow_res

openimage_0922.flo  openimage_0995.flo


In [23]:
!!python SpyNet/pred.py --model sintel-final --one $mod --two $ref --out $flow_res

['/home/din/miniconda3/envs/buna/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608935911/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)',
 '  return F.conv2d(input, weight, bias, self.stride,']

In [27]:
flow = flow_res + "/openimage_0995.flo"
flow_numpy = readFlow(flow)
print(flow_numpy.shape)

(417, 430, 2)


/tmp/ipykernel_5499/3421057023.py:20: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  data = np.fromfile(f, np.float32, count=2 * int(w) * int(h))
/tmp/ipykernel_5499/3421057023.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return np.resize(data, (int(h), int(w), 2))


In [ ]:

img0_numpy = path_to_img_arr(img0_path,False)
img1_numpy = path_to_img_arr(img1_path,False)
C, H, W = img0_numpy.shape
img1_like_numpy  = warp(img0_numpy,flow_numpy)  ## warp via optical flow img1 to img


img1_like = Image.fromarray(img1_like_numpy)
img1_like_path = RESULT + "/openimage_0995_1_like.png"
#img1_like.save(img1_like_path)
diff_show(img1_like_path,img1_path)
